In [1]:
import pandas as pd
import numpy as np

In [102]:
df = pd.read_csv('MyFoodData.csv')
df.columns = ['ID', 'Name', 'Group', 'Carbohydrate', 'Ranking']
min_carb_intake = 10
max_carb_intake = 100
target_avg_rank = 8
fridge_items = 30
df = df[df.Ranking > 1] # removing items with rank less than 1 because in worst case they wouldn't meet avg
df = df[df.Group != 'Fats and Oils'] # this is an ingredient of foods, no one eats this by itself
# sanity check that all inputs are valid

In [65]:
fridge = df.sample(n=fridge_items, replace=False, ignore_index=True)
fridge = fridge.sort_values(by=['Ranking'], ascending=False)

#initializing a list of beverages 
drinks = fridge[(fridge['Group'] == 'Beverages')]

In [113]:
fridge.iloc[2,3] > 8

True

In [107]:
fridge_n = fridge[['ID','Carbohydrate','Ranking']].to_numpy()
drinks_n = drinks[['ID','Carbohydrate','Ranking']].to_numpy()
menu = np.zeros((1,3), dtype='int64')
# menu = np.vstack((fridge_n,fridge_n[0,:]))
fridge

,ID,Name,Group,Carbohydrate,Ranking
0,174633,Quaker Mothers Graham Bumpers,Breakfast Cereals,89,10
23,781321,Infant Formula Powder Made With Tap Water (Sim...,Baby Foods,7,10
20,170253,Potato Chips Without Salt Reduced Fat,Snacks,68,10
19,782945,Meat Loaf Puerto Rican Style,Meats,11,10
12,783902,Gravy Or Sauce Poultry-Based From Puerto Rican...,Soups and Sauces,5,9
15,789439,Coffee Cappuccino Decaffeinated,Beverages,3,9
29,173182,Rich Chocolate Powder,Beverages,93,9
14,168336,Pork Cured Ham -- Water Added Shank Bone-In Se...,Meats,1,8
8,169449,Beef Variety Meats And By-Products Kidneys Raw,Meats,0,8
27,171162,Soup Chunky Beef Canned Ready-To-Serve,Soups and Sauces,10,8


In [111]:
valid_menu = False
for idx in range(drinks_n.shape[0]):
    if drinks_n.size == 0:
        print('there is no valid menu because there are no drinks')
        break
    else:
        if drinks_n[idx,1] < max_carb_intake and valid_menu is False: # checking if carbs in item are at least smaller than max
            menu[0] = drinks_n[idx]
            for idx2 in range(fridge_n.shape[0]): # getting menu item #2
                if fridge_n[idx2,0] not in menu:
                    average_rank = ((menu[:,2].sum() + fridge_n[idx2,2])/(menu.shape[0] + 1))
                    if average_rank >= 8:
                        total_carbs = menu[:,1].sum() + fridge_n[idx2,1]
                        if total_carbs >= min_carb_intake and total_carbs <= max_carb_intake:
                            menu = np.vstack((menu,fridge_n[idx2,:]))
                            valid_menu = True
                            print('Total carbs: ',menu[:,1].sum())
                            break
                        elif total_carbs < min_carb_intake and valid_menu is False: # getting menu item #3
                            for idx3 in range(fridge_n.shape[0]):
                                if fridge_n[idx3,0] not in menu:
                                    average_rank = ((menu[:,2].sum() + fridge_n[idx3,2])/(menu.shape[0] + 1))
                                    if average_rank >= 8:
                                        total_carbs = menu[:,1].sum() + fridge_n[idx3,1]
                                        if total_carbs >= min_carb_intake and total_carbs <= max_carb_intake:
                                            menu = np.vstack((menu,fridge_n[idx3,:]))
                                            valid_menu = True
                                            print(menu)
                                            print('Total carbs: ',menu[:,1].sum())
                                            break
                                        elif total_carbs < min_carb_intake and valid_menu is False: # getting menu item #4
                                            for idx4 in range(fridge_n.shape[0]):
                                                if fridge_n[idx4,0] not in menu:
                                                    average_rank = ((menu[:,2].sum() + fridge_n[idx4,2])/(menu.shape[0] + 1))
                                                    if average_rank >= 8:
                                                        total_carbs = menu[:,1].sum() + fridge_n[idx4,1]
                                                        if total_carbs >= min_carb_intake and total_carbs <= max_carb_intake:
                                                            menu = np.vstack((menu,fridge_n[idx4,:]))
                                                            valid_menu = True
                                                            print(menu)
                                                            print('Total carbs: ',menu[:,1].sum())
                                                            break
        
        elif not valid_menu:
            print('there is no valid menu')

Total carbs:  100


In [151]:
if drinks_n[ix,0] in drinks_n:
    print('yes')
else :
    print('no')
    
if round((menu[:,2].sum() + drinks_n[idx,2])/(menu.shape[0]+1)) >= 8:

yes


In [170]:
fridge_n

array([[170440,     20,     10],
       [788093,      5,      7],
       [783111,     14,      2]], dtype=int64)